# Compute charged defect potential alignment

Implements the method described in the following papers:

Freysoldt et al. Phys. Rev. Lett. 102, 016402 (2009)

Freysoldt et al. Phys. Status Solidi B 248 (2011)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import erf

3D Gaussian model charge density

In [ ]:
L = 16.8/2.0 #angstrom
N = 500
Q = 3.
sigma = 1.0*0.53 #ang

nx, ny, nz = (N, N, N)
x = np.linspace(-(4.0*L)/3.0, (2.0*L)/3.0, nx)
y = np.linspace(-(4.0*L)/3.0, (2.0*L)/3.0, ny)
z = np.linspace(-(4.0*L)/3.0, (2.0*L)/3.0, nz)
xv, yv, zv= np.meshgrid(x, y, z)

# 3d real charge density
RHO = Q*np.exp(-((xv)**2 + (yv)**2 + (zv)**2)/2/sigma**2) / sigma**3 / np.sqrt((2*np.pi)**3) #ang^-3

plot charge density, x-y average along z

In [ ]:
RHOz = []
for ll in range(nz):
    RHOz.append(np.sum(RHO[:,:,ll])/ny/nx)
    
plt.plot(RHOz, 'r')

charge density fft --> reciprocal space

In [ ]:
RHOG = np.fft.rfftn(RHO) 

# rfftn (a[, s, axes]) compute the N-dimensional discrete Fourier Transforma for real input
# syntax : numpy.fft.rfftn
# fft (a[, n, axis]) compute the one-dimensional discrete Fourier Transform
# rfft (a, [, n, axis]) compute the one-dimensional discrete Fourier Transform for real input

Gx = np.linspace(0, 2*np.pi/(2*L), nx/2+1)
Gy = np.linspace(0, 2*np.pi/(2*L), ny/2+1)
Gz = np.linspace(0, 2*np.pi/(2*L), nz/2+1)
Gxv, Gyv, Gzv= np.meshgrid(Gx, Gy, Gz)


solve 3D poisson eqn in reciprocal space

In [ ]:
VG = np.zeros((len(Gx), len(Gy), len(Gz)), dtype=complex)
#VG[1:] = RHOG[1:]/G[1:]**2

for i in range(len(Gxv)):
    for j in range(len(Gyv)):
        for k in range(len(Gzv)):
            G2 = Gxv[i,j,k]**2 + Gyv[i,j,k]**2 + Gzv[i,j,k]**2
            if G2 > 1e-6:
                VG[i,j,k] = RHOG[i,j,k]/G2 

3D pot in real space

In [ ]:
V2 = np.fft.irfftn(VG) / (2*L)##CHECK ## since 3D potential varies as 1/r it seems more reasonable to normalize by L than L**3
# irfftn (a[, s, axes]) compute the inverse of the N-dimensional FFT of real input
# syntax: numpy.fft.irfftn
#V2 = np.fft.irfftn(VG)/N


x-y average along z

In [ ]:
V2z = []
for jj in range(nz):
    V2z.append(np.sum(V2[:,:,jj])/ny/nx)

In [ ]:
plt.plot(V2z)

In [ ]:
from pymatgen.io.vaspio.vasp_output import Locpot

fin1 = 'LOCPOT-charge' #charged defect
fin2 = 'LOCPOT-bulk' #bulk
lpot1 = Locpot.from_file(fin1)
lpot2 = Locpot.from_file(fin2)
lpot_z1 = lpot1.get_average_along_axis(2)
lpot_z2 = lpot2.get_average_along_axis(2)
lpot_z = lpot_z1 - lpot_z2


#fin3 = 'LOCPOT-neutral' #neutral
#lpot3 = Locpot.from_file(fin3)
#lpot_z3 = lpot3.get_average_along_axis(2)

#from pymatgen.io.vaspio.vasp_output import Chgcar

#fin1 = 'CHGCAR-charge' #charged defect
#fin2 = 'CHGCAR-neutral' #neutral
#chg1 = Chgcar.from_file(fin1)
#chg2 = Chgcar.from_file(fin2)
#chg_z1 = chg1.get_average_along_axis(2)
#chg_z2 = chg2.get_average_along_axis(2)

In [ ]:
V2z = np.array(V2z)

# unit converstion : eV --> V
#k = 1/4*np.pi*epsilon_0 = 9 x 10^9 Nm^2/C^2
# e = 1.6 x 10^-19 C, epsilon_0 = 1/k = 8.85 x 10^-12 C^2/Nm^2
# 1 Joule = 1 Nm = 1 C.V = 6.24 x 10^18 eV
# RHO/epsilon_0 = 1.6 x 10^-19 C / 8.85 x 10^-12 C^2/Nm^2 (C.V x 10^10 Ang)
To_V = 180.79 
n = len(lpot_z1)
x1 = lpot1.get_axis_grid(2)
x1 = np.array(x1)

Plot model and dft electrostatic potential

In [ ]:
# V2z (model potential) after dividing by conversion factor to V
V2z_corr = -V2z/180.79
x_shift = x + 11.25
plt.plot(x_shift, V2z_corr, 'r',label='V-Model')

# DFT potential after correcting for potential due to monopole corection 
# whcih is V^mono_corr = alpha*q/epsilon*length
lpot_z_corr = -lpot_z - (-0.047)
plt.plot(x1 , lpot_z_corr, 'k',label='V-DFT')
plt.grid()
plt.xlabel('Z-axis (A)')
plt.ylabel('Potential (V)')
plt.legend(bbox_to_anchor=(1.4,1))

In [ ]:
Vpot = []
x2 = []
for i,xi in enumerate(x1):
    for j,xj in enumerate(x_shift):
        if abs(xj - xi) < 1e-2:
            #print xi, xj
            x2.append(xj)
            Vpot.append(lpot_z_corr[i] - (V2z_corr[j]))
plt.plot(x2,Vpot, 'b',label='Difference')
plt.hlines(0, min(x2), max(x2), 'c')
plt.plot(x_shift, V2z_corr, 'r',label='V-Model')
plt.plot(x1 , lpot_z_corr, 'k',label='V-DFT')
plt.xlabel('Z-axis (A)')
plt.ylabel('Potential (V)')
plt.grid()
plt.legend(bbox_to_anchor=(0.35,0.35))
plt.savefig('3D_DFT-Model_potential.pdf',format='pdf')
plt.show()